## Installations

Installed bs4 (beautiful soup for pulling data out of HTML and XML files.)
Installed contractions to perform as many contractions as possible on our dataset.
Installed gensim to work with Word2vec feature

In [1]:
# ! pip install bs4
# ! pip install contractions
# ! pip install gensim
# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz

In [2]:
# Importing necessary libraries

import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
import re
import copy
import sklearn
from nltk.tokenize import word_tokenize
import contractions
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ayan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ayan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Ayan\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ayan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Getting dataset and displaying first 5 rows
df= pd.read_csv('./data.tsv', sep='\t', dtype=str, lineterminator='\n', error_bad_lines=False)
df.head()

C:\Users\Ayan\AppData\Local\Temp\ipykernel_496\3343637191.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df= pd.read_csv('./data.tsv', sep='\t', dtype=str, lineterminator='\n', error_bad_lines=False)
Skipping line 10093: expected 15 fields, saw 22
Skipping line 31965: expected 15 fields, saw 22
Skipping line 49886: expected 15 fields, saw 22
Skipping line 49905: expected 15 fields, saw 22

Skipping line 67579: expected 15 fields, saw 22
Skipping line 75367: expected 15 fields, saw 22
Skipping line 92462: expected 15 fields, saw 22
Skipping line 105041: expected 15 fields, saw 22
Skipping line 109697: expected 15 fields, saw 22
Skipping line 121931: expected 15 fields, saw 22

Skipping line 139492: expected 15 fields, saw 22
Skipping line 158729: expected 15 fields, saw 22
Skipping line 165784: expected 15 fields, saw 22
Skipping line 176996: expected 15 fields, saw 22
Skipping line 18292

Skipping line 1979093: expected 15 fields, saw 22
Skipping line 1982997: expected 15 fields, saw 22
Skipping line 1992924: expected 15 fields, saw 22
Skipping line 1996161: expected 15 fields, saw 22
Skipping line 2003175: expected 15 fields, saw 22
Skipping line 2024153: expected 15 fields, saw 22
Skipping line 2026345: expected 15 fields, saw 22

Skipping line 2041159: expected 15 fields, saw 22
Skipping line 2042954: expected 15 fields, saw 22
Skipping line 2044244: expected 15 fields, saw 22
Skipping line 2047949: expected 15 fields, saw 22
Skipping line 2051022: expected 15 fields, saw 22
Skipping line 2052365: expected 15 fields, saw 22
Skipping line 2064460: expected 15 fields, saw 22
Skipping line 2077010: expected 15 fields, saw 22
Skipping line 2083893: expected 15 fields, saw 22

Skipping line 2097514: expected 15 fields, saw 22
Skipping line 2100479: expected 15 fields, saw 22
Skipping line 2103183: expected 15 fields, saw 22
Skipping line 2108608: expected 15 fields, saw 2

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,1797882,R3I2DHQBR577SS,B001ANOOOE,2102612,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5,0,0,N,Y,Five Stars,"Love this, excellent sun block!!",2015-08-31
1,US,18381298,R1QNE9NQFJC2Y4,B0016J22EQ,106393691,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5,0,0,N,Y,Thank you Alba Bontanica!,The great thing about this cream is that it do...,2015-08-31
2,US,19242472,R3LIDG2Q4LJBAO,B00HU6UQAG,375449471,"Elysee Infusion Skin Therapy Elixir, 2oz.",Beauty,5,0,0,N,Y,Five Stars,"Great Product, I'm 65 years old and this is al...",2015-08-31
3,US,19551372,R3KSZHPAEVPEAL,B002HWS7RM,255651889,"Diane D722 Color, Perm And Conditioner Process...",Beauty,5,0,0,N,Y,GOOD DEAL!,I use them as shower caps & conditioning caps....,2015-08-31
4,US,14802407,RAI2OIG50KZ43,B00SM99KWU,116158747,Biore UV Aqua Rich Watery Essence SPF50+/PA+++...,Beauty,5,0,0,N,Y,this soaks in quick and provides a nice base f...,This is my go-to daily sunblock. It leaves no ...,2015-08-31


In [4]:
# Keeping only reviews and ratings
df = df[['review_body','star_rating']]

# removing rows with null values
df=df.dropna()
df=df.reset_index(drop=True)

# Changing rating field datatype to int
df['star_rating']=df['star_rating'].astype(int)

# Rename to rating and reviews for easiness
df = df.rename(columns={'review_body': 'review', 'star_rating': 'rating'})

In [5]:
# Making a balanced dataset of of 100,000 rows of 20,000 rows for each rating (1-5)

k=3 # random seed value
df1 = df.query("rating == 1").sample(n=20000,random_state=k)
df2 = df.query("rating == 2").sample(n=20000,random_state=k)
df3 = df.query("rating == 3").sample(n=20000,random_state=k)
df4 = df.query("rating == 4").sample(n=20000,random_state=k)
df5 = df.query("rating == 5").sample(n=20000,random_state=k)
df=pd.concat([df1, df2, df3, df4, df5])
df.rating.value_counts()

1    20000
2    20000
3    20000
4    20000
5    20000
Name: rating, dtype: int64

# Data Cleaning

In [6]:
# Remove  all HTML and URLS from reviews
df['review']=df['review'].apply(lambda x: BeautifulSoup(x).get_text())
df['review']=df['review'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

C:\Users\Ayan\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\Ayan\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [7]:
# perform contractions
# Used the contractions package to perform contractions
df.review=df.review.astype(str)
df['review'] = df['review'].apply(lambda x: contractions.fix(x))

In [8]:
# Remove non alphabetical characters
df['review']=df['review'].str.replace('[^a-zA-Z ]', '')

C:\Users\Ayan\AppData\Local\Temp\ipykernel_496\3446312180.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review']=df['review'].str.replace('[^a-zA-Z ]', '')


In [9]:
# Remove leading and trailing spaces
df['review'] = df['review'].str.strip()

In [10]:
df.head()

,review,rating
198266,Note the blurry picture That is to disguise t...,1
2119369,Too small and thin consistency,1
672827,This was the worst purchase I have ever made T...,1
2249575,NEVER GOT MY ORDER,1
2235550,Do Not waste your time Mine never came on and...,1


## Saving cleaned dataset to use later

In [11]:
# Save to csv file
df.to_csv("cleanedDataset.csv", index=False)

## Loading cleaned dataset and cleaning some more in case of null values

In [12]:
dfc = pd.read_csv("cleanedDataset.csv")
dfc = dfc.loc[:, ~dfc.columns.str.match('Unnamed')] # Remove "Unnamed" index column which forms in saved csv files

In [13]:
# Sometimes null values comeup after loading (mostly for csv files stored in google drive)
dfc['review'].isnull().values.sum()

40

In [14]:
# Remove null values in case formed
dfc=dfc.dropna()
dfc=dfc.reset_index(drop=True)

In [15]:
# dfc['review']=dfc['review'].astype(str)
# Dividing into X and Y numpy arrays based on reviews and labels
X = dfc['review'].to_numpy()
Y = dfc['rating'].to_numpy()

In [16]:
X

array(['Note the blurry picture  That is to disguise the fact that these are tiny travel sizes  Yes I thought it seemed to good to be true but as an extensive online shopper I have found numbers of amazing deals in the past This is not  Basically I paid eight bucks for  dollars worth of conditioner  Good product but not from this seller',
       'Too small and thin consistency',
       'This was the worst purchase I have ever made The shapes do not stay They are completely blurry or just blobs',
       ...,
       'Applies smoothly feels wonderful Makeup does not crumble or fade It is slightly tinted but not so much that you really have to worry about what color it is I do not think it is thick enough to matter',
       'Genuine Product Shipped in time It smells great Perfect gift to your datespouse',
       'Not much to say about this product except for the fact that it works and keeps an expensive electric razor running like new  I have owned Norelco and Braun razors for the last  ye

In [17]:
# Converting review which is a string to a list of tokenized words for training our own word2vec model later
from nltk.tokenize import word_tokenize
for i in range(len(X)):
    X[i] = [t for t in word_tokenize(X[i])]

In [18]:
X

array([list(['Note', 'the', 'blurry', 'picture', 'That', 'is', 'to', 'disguise', 'the', 'fact', 'that', 'these', 'are', 'tiny', 'travel', 'sizes', 'Yes', 'I', 'thought', 'it', 'seemed', 'to', 'good', 'to', 'be', 'true', 'but', 'as', 'an', 'extensive', 'online', 'shopper', 'I', 'have', 'found', 'numbers', 'of', 'amazing', 'deals', 'in', 'the', 'past', 'This', 'is', 'not', 'Basically', 'I', 'paid', 'eight', 'bucks', 'for', 'dollars', 'worth', 'of', 'conditioner', 'Good', 'product', 'but', 'not', 'from', 'this', 'seller']),
       list(['Too', 'small', 'and', 'thin', 'consistency']),
       list(['This', 'was', 'the', 'worst', 'purchase', 'I', 'have', 'ever', 'made', 'The', 'shapes', 'do', 'not', 'stay', 'They', 'are', 'completely', 'blurry', 'or', 'just', 'blobs']),
       ...,
       list(['Applies', 'smoothly', 'feels', 'wonderful', 'Makeup', 'does', 'not', 'crumble', 'or', 'fade', 'It', 'is', 'slightly', 'tinted', 'but', 'not', 'so', 'much', 'that', 'you', 'really', 'have', 'to', 'wor

In [19]:
# To lower RAM usage so that  the session doesn't crash
df=0
dfc=0

# 2. Word Embeddings

## a) Using pretrained model

In [20]:
# Loading pretrained “word2vec-google-news-300” Word2Vec model
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [ ]:
# number of words (3,000,000) of vocabulary for pretrained model. Runtime always disconnected when tried to display all the words.
len(wv.wv.vocab.keys())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


3000000

Checking semantic similarities using three examples

In [ ]:
# Example 1
# husband - man + woman = wife
# Finding cosine similarity using numpy
from numpy import dot
from numpy.linalg import norm
vec_hus = wv['husband']
vec_man = wv['man']
test_vec = (vec_hus - vec_man) + wv['woman']
wife_vec = wv['wife']
result = dot(test_vec, wife_vec)/(norm(test_vec)*norm(wife_vec))
print(result)

0.7332698


In [ ]:
# Example 2
wv.similarity('excited','thrilled')

0.8411032

In [ ]:
# Example 3
wv.similarity('appropriate','suitable')

0.57388663

## b) Using trained model

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=X, size=300, window=11, min_count=10, workers=4)

In [ ]:
# number of words (7475) of vocabulary for our trained model
len(model.wv.vocab.keys())

7475

In [ ]:
# Example 1
# husband - man + woman = wife
# Finding cosine similarity using numpy
from numpy import dot
from numpy.linalg import norm
vec_hus = model['husband']
vec_man = model['man']
test_vec = (vec_hus - vec_man) + model['woman']
wife_vec = model['wife']
result = dot(test_vec, wife_vec)/(norm(test_vec)*norm(wife_vec))
print(result)

0.70438445


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [ ]:
# Example 2
model.similarity('excited','thrilled')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


0.55498016

In [ ]:
# Example 3
model.similarity('appropriate','suitable')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


0.8048134

# Questions:
What do you conclude from comparing vectors
generated by yourself and the pretrained model? Which of the Word2Vec
models seems to encode semantic similarities between words better?
# Answers:
It seems that the pretrained Word2vec model encodes semantic similarities better than my trained model.I think the pretrained model seems to encode similarities better than my model, as it has gained a lot of information from all the words it has been trained on.<br>
I can see that  the there are a lot more vocabulary keys generated by the pretrained word2vec model. This is because it is trained on a very large dataset for a long period of time which makes it a more accurate model for getting word embeddings for many different words. Word2Vec is trained on the Google News dataset (about 100 billion words). It has several use cases such as Recommendation Engines, Knowledge Discovery, and the one we are using it for, Text Classification.
<br>


# 3. Simple Models

### TF-IDF

In [ ]:
# Converting reviews with tokenized words back to sentences for TF-IDF
Xtf = copy.deepcopy(X)
for i in range(len(X)):
  Xtf[i] = ' '.join(Xtf[i])

In [ ]:
# Back to a sentence
Xtf[0]

'not happy with the present way to small they should really put the size as is not make it times bigger in the picture'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvectorizer = TfidfVectorizer(analyzer='word')
Xtf = tfidfvectorizer.fit_transform(Xtf)

In [ ]:
# Train Test Split
Xtf_train, Xtf_test, Ytf_train, Ytf_test = train_test_split(Xtf, Y, test_size=0.2, random_state=42)

In [ ]:
Xtf_train.shape

(79967, 46508)

### Word2vec

In [ ]:
Xw2v_mean = copy.deepcopy(X)
words_in_wv = set(wv.wv.vocab.keys()) # Making a set of all the word2vec words in the pretrained model

# taking average of all word vectors in a review only if their word vectors exist in the pretrained model
for i in range(len(Xw2v_mean)):
  Xw2v_mean[i] = np.mean(np.array([wv[w] for w in Xw2v_mean[i] if w in words_in_wv]), axis = 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# All word have 300 different magnitude values in their vectors or embeddings
# Finding out number of reviews which have number of vector values not equal to 300 
len(Xw2v_mean[[y.size != 300 for y in Xw2v_mean]])

60

In [ ]:
# Removing indices which have less than 300 vector values
indices = np.array([y.size==300 for y in Xw2v_mean])
Xw2v_mean = Xw2v_mean[indices]
Yw2v_mean = Y[indices] # getting labels with those indices

In [ ]:
# Checking again
len(Xw2v_mean[[y.size != 300 for y in Xw2v_mean]])

0

In [ ]:
Xw2v_mean.shape # earlier it was (99959,)

(99899,)

In [ ]:
# Train Test Split
Xw2v_train, Xw2v_test, Yw2v_train, Yw2v_test = train_test_split(Xw2v_mean, Yw2v_mean, test_size=0.2, random_state=42)

In [ ]:
Xw2v_train[0].shape

(300,)

## Perceptron

In [ ]:
# Getting results with TF-IDF features
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report
clf_tf = Perceptron(tol=1e-3, random_state=3)
clf_tf.fit(Xtf_train, Ytf_train)
y_pred = clf_tf.predict(Xtf_test)
print(classification_report(Ytf_test, y_pred, target_names=["1","2","3","4","5"], output_dict = False))

              precision    recall  f1-score   support

           1       0.46      0.62      0.53      4013
           2       0.32      0.23      0.27      4050
           3       0.33      0.32      0.33      4035
           4       0.38      0.36      0.37      3977
           5       0.56      0.59      0.57      3917

    accuracy                           0.42     19992
   macro avg       0.41      0.42      0.41     19992
weighted avg       0.41      0.42      0.41     19992



In [ ]:
# Getting results with Word2vec features
clf_w2v = Perceptron(tol=1e-3, random_state=3)
clf_w2v.fit(Xw2v_train.tolist(), Yw2v_train) # converting to nparray of list as it was an np array of objects with datatype 'object' which needs to be float
y_pred = clf_w2v.predict(Xw2v_test.tolist())
print(classification_report(Yw2v_test, y_pred, target_names=["1","2","3","4","5"], output_dict = False))

              precision    recall  f1-score   support

           1       0.58      0.37      0.45      4012
           2       0.36      0.29      0.32      4049
           3       0.40      0.05      0.08      3999
           4       0.31      0.04      0.06      4073
           5       0.28      0.94      0.43      3847

    accuracy                           0.33     19980
   macro avg       0.39      0.34      0.27     19980
weighted avg       0.39      0.33      0.27     19980



## SVM

In [ ]:
# Getting results with Tf-IDF features
from sklearn.svm import LinearSVC
SVM_tf = LinearSVC(random_state = 0)
SVM_tf.fit(Xtf_train,Ytf_train)
y_pred = SVM_tf.predict(Xtf_test)
print(classification_report(Ytf_test, y_pred, target_names=["1","2","3","4","5"]))

              precision    recall  f1-score   support

           1       0.56      0.66      0.60      4013
           2       0.39      0.32      0.35      4050
           3       0.42      0.39      0.41      4035
           4       0.45      0.41      0.43      3977
           5       0.63      0.73      0.68      3917

    accuracy                           0.50     19992
   macro avg       0.49      0.50      0.49     19992
weighted avg       0.49      0.50      0.49     19992



In [ ]:
# Getting results with Word2vec features
SVM_w2v = LinearSVC(random_state = 0)
SVM_w2v.fit(Xw2v_train.tolist(),Yw2v_train)
y_pred = SVM_w2v.predict(Xw2v_test.tolist())
print(classification_report(Yw2v_test, y_pred, target_names=["1","2","3","4","5"]))

              precision    recall  f1-score   support

           1       0.50      0.71      0.59      4012
           2       0.39      0.25      0.30      4049
           3       0.40      0.38      0.39      3999
           4       0.46      0.30      0.36      4073
           5       0.57      0.76      0.65      3847

    accuracy                           0.48     19980
   macro avg       0.46      0.48      0.46     19980
weighted avg       0.46      0.48      0.46     19980



Accuracy values:
<br>
Perceptron:<br>•
Tf-IDF : 0.42<br>•
W2V : 0.33
<br>
SVM:<br>•
Tf-IDF : 0.50<br>•
W2V : 0.48

# Question
What do you conclude from comparing performances for the models trained using the two different feature types (TF-IDF and your trained Word2Vec features)?
# Answer
Based on my results TF-IDF performed better than Word2vec for Perceptron model, but performed about the same for SVM model. Also, SVM for W2V model took a long time to train. Its probably because it took long to decide the margin, only if that affects the training time. Overall SVM performed better than Perceptron. As perceptron only does well with linearly separable datapoints.<br>
There are 46508 features per review in TF-IDF, whereas there are 300 features in W2V, which we got from averaging all the words in a review. The reason why W2V performed worse could be that the averaging of word vector values makes the feature lose information which connects it to the label and feeding this data to a simple model sucha as a Perceptron won't give a good accuracy. A RNN could produce better results for this five class classification task. Also, TF-IDF is a statistical measure that we can apply to terms in a document and then use that to form a vector. So, its method is much more better as it is specific to this dataset. Whereas, W2V word embeddings are coming from a pretrained vector which has a large amount of information which may not be specific  to this dataset.

In [ ]:
# To lower RAM usage so that  the session doesn't crash
Xtf_train=0
Xtf_test=0
Ytf_train=0
Ytf_test=0
Xtf=0
Xw2v_train=0
Xw2v_test=0
Yw2v_train=0
Yw2v_test=0

# Feedforward Neural Networks

## a) Averaged vectors for all reviews

In [ ]:
# In case session crashed
# uncomment below lines and run this cell
# if not ignore cell

# # Loading pretrained “word2vec-google-news-300” Word2Vec model
# import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')

# import pandas as pd
# import numpy as np
# import copy
# dfc = pd.read_csv("cleanedDataset.csv")
# dfc = dfc.loc[:, ~dfc.columns.str.match('Unnamed')] # Remove "Unnamed" index column which forms in saved csv files

# # Remove null values in case formed
# dfc=dfc.dropna()
# dfc=dfc.reset_index(drop=True)

# # dfc['review']=dfc['review'].astype(str)
# # Dividing into X and Y numpy arrays based on reviews and labels
# X = dfc['review'].to_numpy()
# Y = dfc['rating'].to_numpy()

# # Converting review which is a string to a list of tokenized words for training our own word2vec model later
# from nltk.tokenize import word_tokenize
# for i in range(len(X)):
#     X[i] = [t for t in word_tokenize(X[i])]

In [ ]:
# Taking mean w2v features from previous question and going straight to split

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(Xw2v_mean, Yw2v_mean, test_size=0.2, random_state=42)

In [ ]:
# converting to nparray of list as it was an np array of objects with datatype 'object' which needs to be float
def arrayToList(my_array):
  return np.array(my_array.tolist())

train_x = arrayToList(train_x)
test_x = arrayToList(test_x)

In [ ]:
# reducing labels from 1-5 to 0-4
# Run this only once or you will end up subtracting two times
test_y = test_y - 1
train_y = train_y - 1


# # Code to check unique counts per label
# unique, counts = np.unique(test_y, return_counts=True)
# dict(zip(unique, counts))

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# create Tensor Dataset
# train_data=TensorDataset(torch.cuda.FloatTensor(train_x), torch.cuda.LongTensor(train_y))
# test_data=TensorDataset(torch.cuda.FloatTensor(test_x), torch.cuda.LongTensor(test_y))

# Creating non cuda TensorDataset as CUDA didn't provide type of error when it occurs. Thus, was very difficult to debug.
train_data=TensorDataset(torch.FloatTensor(train_x), torch.LongTensor(train_y))
test_data=TensorDataset(torch.FloatTensor(test_x), torch.LongTensor(test_y))

# dataloader
# Using 256 batch size as per tutorial
train_batch_size=256

train_loader=DataLoader(train_data, batch_size=train_batch_size, shuffle=True)


In [ ]:
# MLP class

import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # number of hidden nodes in each layer
        hidden_1 = 50
        hidden_2 = 10
        # linear layer (300 -> hidden_1)
        self.fc1 = nn.Linear(300, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # linear layer (n_hidden -> 5)
        self.fc3 = nn.Linear(hidden_2, 5)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        # self.dropout = nn.Dropout(0.2)
        # initialize weights
        # self.weight_init()
        # was not learning so commented out weight initialization
        # Pytorch auto updates weights and initializing step above was changing it back to initial weights

    def forward(self, x):
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        # x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        # x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        return x

    def weight_init(self):
        # Initialize weights
        nn.init.zeros_(self.fc1.weight)
        nn.init.zeros_(self.fc2.weight)
        nn.init.zeros_(self.fc3.weight)
        nn.init.ones_(self.fc1.bias)
        nn.init.ones_(self.fc2.bias)
        nn.init.ones_(self.fc3.bias)
        

model = MLP()
print(model)

MLP(
  (fc1): Linear(in_features=300, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=5, bias=True)
)


## Reason for using two accuracies
Also, I am gathering two accuracies as I was unsure about my calculation of accuracy.
The true accuracy is just named that way to differentiate between the first one and second one. Both are correct.<br>
Validation accuracy is gathering correct predictions and dividing it by amount of data in a epoch.<br>
The True accuracy is using sklearn metric to get accuracy per batch and in the end is averaging it over all the batches for that epoch.<br>
Both give the same result

In [ ]:
test_batch_size=256 # using 256 for test batch size for now to use it as a validation set
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)

# Also, I am gathering two accuracies as I was unsure about my calculation of accuracy
# The true accuracy is just named that way to differentiate between the first one and second one. Both are correct
# Validation accuracy is gathering 

# for accuracy
from sklearn.metrics import accuracy_score

# specify loss function (categorical cross-entropy)
loss_fn = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# training params
epochs = 500

# initialize device
# device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# model.cuda()

# train for some number of epochs 150
for e in range(epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0
  
  model.train()
  # batch loop
  for inputs, labels in train_loader:
    # inputs=inputs.cuda()
    # labels=labels.cuda()
    # labels = labels.type(torch.LongTensor)   # casting to long
    # inputs, labels = inputs.to(device), labels.to(device)
    # zero accumulated gradients
    optimizer.zero_grad()
    # get the output from the model
    output = model(inputs)
    # calculate the loss and perform backprop
    loss = loss_fn(output, labels)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer.step()
    
    # update running training loss
    train_loss += loss.item()*inputs.size(0)

  model.eval() # prep model for evaluation
  num_correct = 0 # initialize num_correct for correct predictions
  epoch_acc=0
  count=0
  for inputs, labels in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(inputs)
    # calculate the loss
    loss = loss_fn(output, labels)
    # update running validation loss 
    valid_loss += loss.item()*inputs.size(0)
    # convert output probabilities to predicted class (1-5)
    # pred = torch.argmax(output, dim=1, keepdim=True)
    pred = output.argmax(dim=1, keepdim=True)
    # pred = torch.round(pred)  # rounds to the nearest integer
    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    # correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)
    epoch_acc += accuracy_score(pred, labels)
    count += 1

  # print training/validation statistics 
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader.dataset)
  valid_loss = valid_loss/len(test_loader.dataset)

  # accuracy over all test data
  valid_acc = num_correct/len(test_loader.dataset)
  true_acc = epoch_acc/count

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Acc: {:.6f} \tTrue Acc: {:.6f}'.format(
      e+1, 
      train_loss,
      valid_loss,
      valid_acc,
      true_acc
      ))

Epoch: 1 	Training Loss: 1.620935 	Validation Loss: 1.616416 	Validation Acc: 0.192543 	True Acc: 0.192230
Epoch: 2 	Training Loss: 1.612387 	Validation Loss: 1.611551 	Validation Acc: 0.192543 	True Acc: 0.193236
Epoch: 3 	Training Loss: 1.610055 	Validation Loss: 1.610068 	Validation Acc: 0.192643 	True Acc: 0.192329
Epoch: 4 	Training Loss: 1.609388 	Validation Loss: 1.609539 	Validation Acc: 0.193093 	True Acc: 0.191769
Epoch: 5 	Training Loss: 1.609161 	Validation Loss: 1.609287 	Validation Acc: 0.192993 	True Acc: 0.192675
Epoch: 6 	Training Loss: 1.609051 	Validation Loss: 1.609169 	Validation Acc: 0.192643 	True Acc: 0.193335
Epoch: 7 	Training Loss: 1.608971 	Validation Loss: 1.609044 	Validation Acc: 0.194745 	True Acc: 0.195411
Epoch: 8 	Training Loss: 1.608896 	Validation Loss: 1.608956 	Validation Acc: 0.199049 	True Acc: 0.196648
Epoch: 9 	Training Loss: 1.608810 	Validation Loss: 1.608852 	Validation Acc: 0.201552 	True Acc: 0.201131
Epoch: 10 	Training Loss: 1.608709 	V

In [ ]:
torch.save(model.state_dict(), 'fnn_model.pt')

In [ ]:
model.load_state_dict(torch.load('fnn_model.pt'))

<All keys matched successfully>

In [ ]:
# changing test_batch_size to 1 to go through all the data and record accuracy over it all
test_batch_size=1
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)


# Checking for accuracy
test_losses = [] # track loss
num_correct = 0

model.eval()
# iterate over test data
for inputs, labels in test_loader:
    # inputs, labels = inputs.cuda(), labels.cuda()
    output = model(inputs)

    # calculate loss
    test_loss = loss_fn(output, labels)
    test_losses.append(test_loss.item())
    # convert output probabilities to predicted class (0-4)
    pred = torch.argmax(output, dim=1)  # rounds to the nearest integer
    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 1.170
Test accuracy: 0.490


## Accuracy value for testing split: 0.49

In [ ]:
# To lower RAM usage so that the session doesn't crash
Xw2v_mean=0
Yw2v_mean=0

## b) Concatenate first 10 words of review

In [ ]:
# In case session crashed
# uncomment below lines and run this cell
# if not ignore cell

# # Loading pretrained “word2vec-google-news-300” Word2Vec model
# import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')

# import pandas as pd
# import numpy as np
# import copy
# dfc = pd.read_csv("cleanedDataset.csv")
# dfc = dfc.loc[:, ~dfc.columns.str.match('Unnamed')] # Remove "Unnamed" index column which forms in saved csv files

# # Remove null values in case formed
# dfc=dfc.dropna()
# dfc=dfc.reset_index(drop=True)

# # dfc['review']=dfc['review'].astype(str)
# # Dividing into X and Y numpy arrays based on reviews and labels
# X = dfc['review'].to_numpy()
# Y = dfc['rating'].to_numpy()

# # Converting review which is a string to a list of tokenized words for training our own word2vec model later
# from nltk.tokenize import word_tokenize
# for i in range(len(X)):
#     X[i] = [t for t in word_tokenize(X[i])]

In [ ]:
# Concatenating first 10 words of a review
Xw2v_con = copy.deepcopy(X)
words_in_wv = set(wv.wv.vocab.keys())
for i in range(len(Xw2v_con)):
  data = Xw2v_con[i]
  tmp = np.array([wv[w] for w in Xw2v_con[i][:10] if w in words_in_wv])
  if(len(tmp)==0):
    tmp = np.zeros((1,300))
  data = np.concatenate(tmp, axis=0)

  if(len(data)<3000):
    data = np.concatenate([data, np.zeros(3000-len(data))])
  Xw2v_con[i] = data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
len(Xw2v_con[[y.size != 3000 for y in Xw2v_con]])

0

In [ ]:
Xw2v_con.shape # same (99959,)

(99959,)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(Xw2v_con, Y, test_size=0.2, random_state=42)

In [ ]:
# converting to nparray of list as it was an np array of objects with datatype 'object' which needs to be float
def arrayToList(my_array):
  return np.array(my_array.tolist())

train_x = arrayToList(train_x)
test_x = arrayToList(test_x)

In [ ]:
# reducing labels from 1-5 to 0-4
test_y = test_y - 1
train_y = train_y - 1

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

#create Tensor Dataset
# train_data=TensorDataset(torch.cuda.FloatTensor(train_x), torch.cuda.LongTensor(train_y))
# test_data=TensorDataset(torch.cuda.FloatTensor(test_x), torch.cuda.LongTensor(test_y))

train_data=TensorDataset(torch.FloatTensor(train_x), torch.LongTensor(train_y))
test_data=TensorDataset(torch.FloatTensor(test_x), torch.LongTensor(test_y))

#dataloader
train_batch_size=256
train_loader=DataLoader(train_data, batch_size=train_batch_size, shuffle=True)


In [ ]:
# MLP class

import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # number of hidden nodes in each layer
        hidden_1 = 50
        hidden_2 = 10
        # linear layer (300 -> hidden_1)
        self.fc1 = nn.Linear(3000, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # linear layer (n_hidden -> 5)
        self.fc3 = nn.Linear(hidden_2, 5)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        # self.dropout = nn.Dropout(0.2)
        # initialize weights
        # self.weight_init()

    def forward(self, x):
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        # x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        # x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        return x

    def weight_init(self):
        # Initialize weights
        nn.init.zeros_(self.fc1.weight)
        nn.init.zeros_(self.fc2.weight)
        nn.init.zeros_(self.fc3.weight)
        nn.init.ones_(self.fc1.bias)
        nn.init.ones_(self.fc2.bias)
        nn.init.ones_(self.fc3.bias)
        

model = MLP()
print(model)

MLP(
  (fc1): Linear(in_features=3000, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=5, bias=True)
)


In [ ]:
test_batch_size=256
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)

# for accuracy
from sklearn.metrics import accuracy_score

# specify loss function (categorical cross-entropy)
loss_fn = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# training params
epochs = 60

# initialize device
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# model.cuda()
# train for some number of epochs
for e in range(epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0
  
  model.train()
  # batch loop
  for inputs, labels in train_loader:
    # inputs=inputs.cuda()
    # labels=labels.cuda()
    # labels = labels.type(torch.LongTensor)   # casting to long
    # inputs, labels = inputs.to(device), labels.to(device)
    # zero accumulated gradients
    optimizer.zero_grad()
    # get the output from the model
    output = model(inputs)
    # calculate the loss and perform backprop
    loss = loss_fn(output, labels)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer.step()
    
    # update running training loss
    train_loss += loss.item()*inputs.size(0)

  model.eval() # prep model for evaluation
  num_correct = 0 # initialize num_correct for correct predictions
  epoch_acc=0
  count=0
  for inputs, labels in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(inputs)
    # calculate the loss
    loss = loss_fn(output, labels)
    # update running validation loss 
    valid_loss += loss.item()*inputs.size(0)
    # print(inputs)
    # print(output, labels)
    # print(output,labels)
    # print(output.dtype,labels.dtype)
    # convert output probabilities to predicted class (1-5)
    # pred = torch.argmax(output, dim=1, keepdim=True)
    pred = output.argmax(dim=1, keepdim=True)
    # pred = torch.round(pred)  # rounds to the nearest integer
    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    # correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)
    epoch_acc += accuracy_score(pred, labels)
    count += 1

  # print training/validation statistics 
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader.dataset)
  valid_loss = valid_loss/len(test_loader.dataset)

  # accuracy over all test data
  valid_acc = num_correct/len(test_loader.dataset)
  true_acc = epoch_acc/count

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Acc: {:.6f} \tTrue Acc: {:.6f}'.format(
      e+1, 
      train_loss,
      valid_loss,
      valid_acc,
      true_acc
      ))

Epoch: 1 	Training Loss: 1.615914 	Validation Loss: 1.612286 	Validation Acc: 0.195928 	True Acc: 0.195593
Epoch: 2 	Training Loss: 1.610020 	Validation Loss: 1.609150 	Validation Acc: 0.195928 	True Acc: 0.196549
Epoch: 3 	Training Loss: 1.608314 	Validation Loss: 1.607956 	Validation Acc: 0.209834 	True Acc: 0.209817
Epoch: 4 	Training Loss: 1.607452 	Validation Loss: 1.607097 	Validation Acc: 0.233093 	True Acc: 0.232809
Epoch: 5 	Training Loss: 1.606637 	Validation Loss: 1.606200 	Validation Acc: 0.247399 	True Acc: 0.246473
Epoch: 6 	Training Loss: 1.605682 	Validation Loss: 1.605131 	Validation Acc: 0.265906 	True Acc: 0.265724
Epoch: 7 	Training Loss: 1.604486 	Validation Loss: 1.603783 	Validation Acc: 0.283964 	True Acc: 0.283574
Epoch: 8 	Training Loss: 1.602952 	Validation Loss: 1.602044 	Validation Acc: 0.288916 	True Acc: 0.287513
Epoch: 9 	Training Loss: 1.600944 	Validation Loss: 1.599732 	Validation Acc: 0.298119 	True Acc: 0.297567
Epoch: 10 	Training Loss: 1.598276 	V

In [ ]:
torch.save(model.state_dict(), 'fnn_b_model.pt')

In [ ]:
model.load_state_dict(torch.load('fnn_b_model.pt'))
test_batch_size=1
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)


# Checking for accuracy
test_losses = [] # track loss
num_correct = 0

model.eval()
# iterate over test data
for inputs, labels in test_loader:
    # inputs, labels = inputs.cuda(), labels.cuda()
    output = model(inputs)

    # calculate loss
    test_loss = loss_fn(output, labels)
    test_losses.append(test_loss.item())
    # print(inputs)
    # print(output, labels)
    # convert output probabilities to predicted class (0-4)
    pred = torch.argmax(output, dim=1)  # rounds to the nearest integer
    # print(pred)
    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    # correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 1.318
Test accuracy: 0.421


## Accuracy for testing split: 0.421

# Question
What do you conclude by comparing accuracy values you obtain with those obtained in the “Simple Models” section?
# Answer
Based on the results of the simple models, I got accuracy values of:<br> TF-IDF: 0.42 (Perceptron) and 0.50 (SVM)
<br>W2V: 0.33 (Perceptron) and 0.48 (SVM)
<br>FNN model a) 0.49 and b) 0.42
<br>I will conclude that the W2V feature usage improved with the FNN model a) and in FNN model b) the concatenated W2V features of the first 10 words do not seem to connect well to the labels. As not all reviews show their sentiment in the first 10 words and some reviews have less than 10 words (which were concatenated with zero value vectors). Due to these I think the model b) didn't run as well as a). Also, Neural Network models are less sensitive with respect to hyperparameters, and training data preparation is more straightforward and systematic. Nowadays, with more computing power FNN perform way better than traditional SVM and Perceptron models.

In [ ]:
# to lower ram usage so that the session doesn't crash
Xw2v_con=0
data_iter=0
train_x=0
test_x=0
train_y=0
test_y=0
test_loader=0
train_loader=0

# Recurrent Neural Networks

## a) RNN 

In [ ]:
# In case session crashed
# uncomment below lines and run this cell

# Loading pretrained “word2vec-google-news-300” Word2Vec model
# import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')

# import pandas as pd
# import numpy as np
# import copy
# dfc = pd.read_csv("cleanedDataset.csv")
# dfc = dfc.loc[:, ~dfc.columns.str.match('Unnamed')] # Remove "Unnamed" index column which forms in saved csv files

# # Remove null values in case formed
# dfc=dfc.dropna()
# dfc=dfc.reset_index(drop=True)

# # dfc['review']=dfc['review'].astype(str)
# # Dividing into X and Y numpy arrays based on reviews and labels
# X = dfc['review'].to_numpy()
# Y = dfc['rating'].to_numpy()

# # Converting review which is a string to a list of tokenized words for training our own word2vec model later
# from nltk.tokenize import word_tokenize
# for i in range(len(X)):
#     X[i] = [t for t in word_tokenize(X[i])]

In [22]:
# Keeping first 20 words in the review as separate arrays
Xw2v_RNN = copy.deepcopy(X)
# words_in_wv = set(wv.wv.vocab.keys())
for i in range(len(Xw2v_RNN)):
  review = Xw2v_RNN[i]

  if(len(review)<20):
    temp = np.array([wv[w] for w in review if w in wv])
  else:
    temp = np.array([wv[w] for w in review[:20] if w in wv])
    
  if len(temp) == 0:
    review = np.zeros((20,300))
  elif len(temp)<20:
    review = np.concatenate([temp, np.zeros((20-len(temp), 300))])
  else:
    review = temp
  Xw2v_RNN[i] = review

MemoryError: Unable to allocate 46.9 KiB for an array with shape (20, 300) and data type float64

In [ ]:
print(len(Xw2v_RNN))

99959


In [ ]:
# To lower RAM usage this one is needed as Xw2v_RNN stores a lot of data
wv=0
X=0
set_words=0

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(Xw2v_RNN, Y, test_size=0.2, random_state=42)

In [ ]:
# To lower RAM usage as Xw2v_RNN stores a lot of data
Xw2v_RNN=0
Y=0

In [ ]:
# converting form array of array to array of list object
def arrayToList(my_array):
  return np.array(my_array.tolist())

train_x = arrayToList(train_x)
test_x = arrayToList(test_x)

In [ ]:
# reducing labels from 1-5 to 0-4
test_y = test_y - 1
train_y = train_y - 1
# run only once

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

#create Tensor Dataset
# train_data=TensorDataset(torch.cuda.FloatTensor(train_x), torch.cuda.LongTensor(train_y))
# test_data=TensorDataset(torch.cuda.FloatTensor(test_x), torch.cuda.LongTensor(test_y))

train_data=TensorDataset(torch.FloatTensor(train_x), torch.LongTensor(train_y))
test_data=TensorDataset(torch.FloatTensor(test_x), torch.LongTensor(test_y))

#dataloader
train_batch_size=256
train_loader=DataLoader(train_data, batch_size=train_batch_size, shuffle=True)

In [ ]:
import torch.nn as nn

class modelRNN(nn.Module):
    def __init__(self):
        super(modelRNN, self).__init__()

        # self.hidden_size = hidden_size

        # self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        # self.i2o = nn.Linear(input_size + hidden_size, output_size)
        # self.softmax = nn.LogSoftmax(dim=1)
        self.rnn = nn.RNN(300, 20, batch_first = True, nonlinearity='relu')
        
        self.fc = nn.Linear(20,5)

    def forward(self, input):
        output = input.view(-1,20,300)
        output, hidden = self.rnn(output)
        output=self.fc(output[:,-1,:])
                
        # hidden = self.i2h(combined)
        # output = self.i2o(combined)
        # output = self.softmax(output)
        return output

model = modelRNN()
print(model)

modelRNN(
  (rnn): RNN(300, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=5, bias=True)
)


In [ ]:
test_batch_size=256
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)

# for accuracy
from sklearn.metrics import accuracy_score

# specify loss function (categorical cross-entropy)
loss_fn = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# training params
epochs = 120

# initialize device
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# model.cuda()
# train for some number of epochs
for e in range(epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0
  
  model.train()
  # batch loop
  for inputs, labels in train_loader:
    # inputs=inputs.cuda()
    # labels=labels.cuda()
    # labels = labels.type(torch.LongTensor)   # casting to long
    # inputs, labels = inputs.to(device), labels.to(device)
    # zero accumulated gradients
    optimizer.zero_grad()
    # get the output from the model
    output = model(inputs)
    # calculate the loss and perform backprop
    loss = loss_fn(output, labels)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer.step()
    
    # update running training loss
    train_loss += loss.item()*inputs.size(0)

  model.eval() # prep model for evaluation
  num_correct = 0 # initialize num_correct for correct predictions
  epoch_acc=0
  count=0
  for inputs, labels in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(inputs)
    # calculate the loss
    loss = loss_fn(output, labels)
    # update running validation loss 
    valid_loss += loss.item()*inputs.size(0)
    # print(inputs)
    # print(output, labels)
    # print(output,labels)
    # print(output.dtype,labels.dtype)
    # convert output probabilities to predicted class (1-5)
    # pred = torch.argmax(output, dim=1, keepdim=True)
    pred = output.argmax(dim=1, keepdim=True)
    # pred = torch.round(pred)  # rounds to the nearest integer
    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    # correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)
    epoch_acc += accuracy_score(pred, labels)
    count += 1

  # print training/validation statistics 
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader.dataset)
  valid_loss = valid_loss/len(test_loader.dataset)

  # accuracy over all test data
  valid_acc = num_correct/len(test_loader.dataset)
  true_acc = epoch_acc/count

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Acc: {:.6f} \tTrue Acc: {:.6f}'.format(
      e+1, 
      train_loss,
      valid_loss,
      valid_acc,
      true_acc
      ))

Epoch: 1 	Training Loss: 1.520409 	Validation Loss: 1.374635 	Validation Acc: 0.365346 	True Acc: 0.367369
Epoch: 2 	Training Loss: 1.327186 	Validation Loss: 1.310352 	Validation Acc: 0.410064 	True Acc: 0.409184
Epoch: 3 	Training Loss: 1.296136 	Validation Loss: 1.291235 	Validation Acc: 0.417517 	True Acc: 0.416073
Epoch: 4 	Training Loss: 1.278782 	Validation Loss: 1.285204 	Validation Acc: 0.421869 	True Acc: 0.422287
Epoch: 5 	Training Loss: 1.263897 	Validation Loss: 1.267683 	Validation Acc: 0.432473 	True Acc: 0.434204
Epoch: 6 	Training Loss: 1.254745 	Validation Loss: 1.261889 	Validation Acc: 0.438575 	True Acc: 0.436412
Epoch: 7 	Training Loss: 1.242926 	Validation Loss: 1.251841 	Validation Acc: 0.441327 	True Acc: 0.441044
Epoch: 8 	Training Loss: 1.233409 	Validation Loss: 1.242190 	Validation Acc: 0.445078 	True Acc: 0.444752
Epoch: 9 	Training Loss: 1.218899 	Validation Loss: 1.244525 	Validation Acc: 0.444628 	True Acc: 0.444785
Epoch: 10 	Training Loss: 1.210522 	V

In [ ]:
torch.save(model.state_dict(), 'rnn_model.pt')

In [ ]:
model.load_state_dict(torch.load('rnn_model.pt'))
test_batch_size=1
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)


# Checking for accuracy
test_losses = [] # track loss
num_correct = 0

model.eval()
# iterate over test data
for inputs, labels in test_loader:
    # inputs, labels = inputs.cuda(), labels.cuda()
    output = model(inputs)

    # calculate loss
    test_loss = loss_fn(output, labels)
    test_losses.append(test_loss.item())
    # print(inputs)
    # print(output, labels)
    # convert output probabilities to predicted class (0-4)
    pred = torch.argmax(output, dim=1)  # rounds to the nearest integer
    # print(pred)
    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    # correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 1.192
Test accuracy: 0.481


## Accuracy for RNN: 0.48

# Question
What do you conclude by comparing accuracy values you obtain with those obtained with feedforward neural network models?
# Answer
For FNN model a) accuracy = 0.49 and for FNN model b) accuracy = 0.42. Now for RNN we got accuracy 0.48. So we can conclude that RNN is performing with similar accuracy as compared to FNN. If we could use more than 20 words in the review we could achieve higher accuracy for RNN. Also, if we could use a different value to fill vector values instead of just using zeros to fill up reviews with less than 20 words.
we could achieve higher accuracy for RNN.<br>
FNN model doesn't seem to obtain the context of the data, long term dependencies of some words and also the information on order of the words is lost. RNN seems to handle this well especially with the historical weight it uses.

## b) **GRU**

In [ ]:
# In case session crashed
# uncomment below lines and run this cell

# # Loading pretrained “word2vec-google-news-300” Word2Vec model
# import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')

# dfc = pd.read_csv("cleanedDataset.csv")
# dfc = dfc.loc[:, ~dfc.columns.str.match('Unnamed')] # Remove "Unnamed" index column which forms in saved csv files

# # Remove null values in case formed
# dfc=dfc.dropna()
# dfc=dfc.reset_index(drop=True)

# # dfc['review']=dfc['review'].astype(str)
# # Dividing into X and Y numpy arrays based on reviews and labels
# X = dfc['review'].to_numpy()
# Y = dfc['rating'].to_numpy()

# # Converting review which is a string to a list of tokenized words for training our own word2vec model later
# from nltk.tokenize import word_tokenize
# for i in range(len(X)):
#     X[i] = [t for t in word_tokenize(X[i])]

In [ ]:
import torch.nn as nn

class modelGRU(nn.Module):
    def __init__(self):
        super(modelGRU, self).__init__()

        # self.hidden_size = hidden_size

        # self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        # self.i2o = nn.Linear(input_size + hidden_size, output_size)
        # self.softmax = nn.LogSoftmax(dim=1)
        self.gru = nn.GRU(300, 20, batch_first = True)
        
        self.fc = nn.Linear(20,5)

    def forward(self, input):
        output = input.view(-1,20,300)
        output, hidden = self.gru(output)
        output=self.fc(output[:,-1,:])
                
        # hidden = self.i2h(combined)
        # output = self.i2o(combined)
        # output = self.softmax(output)
        return output

model = modelGRU()
print(model)

modelGRU(
  (gru): GRU(300, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=5, bias=True)
)


In [ ]:
# batch size  back to 256 from 1 done in RNN
test_batch_size=256
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)

# for accuracy
from sklearn.metrics import accuracy_score

# specify loss function (categorical cross-entropy)
loss_fn = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# training params
epochs = 20

# initialize device
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# model.cuda()
# train for some number of epochs
for e in range(epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0
  
  model.train()
  # batch loop
  for inputs, labels in train_loader:
    # inputs=inputs.cuda()
    # labels=labels.cuda()
    # labels = labels.type(torch.LongTensor)   # casting to long
    # inputs, labels = inputs.to(device), labels.to(device)
    # zero accumulated gradients
    optimizer.zero_grad()
    # get the output from the model
    output = model(inputs)
    # calculate the loss and perform backprop
    loss = loss_fn(output, labels)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer.step()
    
    # update running training loss
    train_loss += loss.item()*inputs.size(0)

  model.eval() # prep model for evaluation
  num_correct = 0 # initialize num_correct for correct predictions
  epoch_acc=0
  count=0
  for inputs, labels in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(inputs)
    # calculate the loss
    loss = loss_fn(output, labels)
    # update running validation loss 
    valid_loss += loss.item()*inputs.size(0)
    # print(inputs)
    # print(output, labels)
    # print(output,labels)
    # print(output.dtype,labels.dtype)
    # convert output probabilities to predicted class (1-5)
    # pred = torch.argmax(output, dim=1, keepdim=True)
    pred = output.argmax(dim=1, keepdim=True)
    # pred = torch.round(pred)  # rounds to the nearest integer
    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    # correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)
    epoch_acc += accuracy_score(pred, labels)
    count += 1

  # print training/validation statistics 
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader.dataset)
  valid_loss = valid_loss/len(test_loader.dataset)

  # accuracy over all test data
  valid_acc = num_correct/len(test_loader.dataset)
  true_acc = epoch_acc/count

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Acc: {:.6f} \tTrue Acc: {:.6f}'.format(
      e+1, 
      train_loss,
      valid_loss,
      valid_acc,
      true_acc
      ))

Epoch: 1 	Training Loss: 1.435480 	Validation Loss: 1.304984 	Validation Acc: 0.412915 	True Acc: 0.414392
Epoch: 2 	Training Loss: 1.258649 	Validation Loss: 1.226030 	Validation Acc: 0.457833 	True Acc: 0.458795
Epoch: 3 	Training Loss: 1.206383 	Validation Loss: 1.195828 	Validation Acc: 0.473689 	True Acc: 0.473513
Epoch: 4 	Training Loss: 1.183951 	Validation Loss: 1.188490 	Validation Acc: 0.475090 	True Acc: 0.476810
Epoch: 5 	Training Loss: 1.169610 	Validation Loss: 1.175286 	Validation Acc: 0.480592 	True Acc: 0.480337
Epoch: 6 	Training Loss: 1.157736 	Validation Loss: 1.168972 	Validation Acc: 0.485294 	True Acc: 0.485941
Epoch: 7 	Training Loss: 1.148287 	Validation Loss: 1.158876 	Validation Acc: 0.487195 	True Acc: 0.487342
Epoch: 8 	Training Loss: 1.140887 	Validation Loss: 1.159673 	Validation Acc: 0.489046 	True Acc: 0.488693
Epoch: 9 	Training Loss: 1.133778 	Validation Loss: 1.154216 	Validation Acc: 0.491397 	True Acc: 0.492451
Epoch: 10 	Training Loss: 1.128104 	V

In [ ]:
torch.save(model.state_dict(), 'gru_model.pt')

In [ ]:
model.load_state_dict(torch.load('gru_model.pt'))
test_batch_size=1
test_loader=DataLoader(test_data, batch_size=test_batch_size, shuffle=True)


# Checking for accuracy
test_losses = [] # track loss
num_correct = 0

model.eval()
# iterate over test data
for inputs, labels in test_loader:
    # inputs, labels = inputs.cuda(), labels.cuda()
    output = model(inputs)

    # calculate loss
    test_loss = loss_fn(output, labels)
    test_losses.append(test_loss.item())
    # print(inputs)
    # print(output, labels)
    # convert output probabilities to predicted class (0-4)
    pred = torch.argmax(output, dim=1)  # rounds to the nearest integer
    # print(pred)
    # compare predictions to true label
    correct_tensor = pred.eq(labels.view_as(pred))
    # correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 1.140
Test accuracy: 0.498


## Accuracy for GRU: 0.498

# Question
What do you conclude by comparing accuracy values you obtain with those obtained using simple RNN?
# Answer
For RNN we got accuracy of 0.48 and for GRU we got accuracy of 0.498. So we can conclude that GRU improved in accuracy by a very tiny amount. Conceptually, RNN faces vanishing/exploding gradients problem, which could affect the way it learns and ultimately lowering its accuracy. Whereas, Gated RNN can learn long term dependencies and can control how much information they pass on. And its usage of tanh function solves the vanishing/exploding gradient problems.